## 测试 rust 版本 czsc 库的使用

----

In [ ]:
from rs_czsc import Event, Signal, CZSC

s1 = Signal(key="30分钟_D1_表里关系V230101", value="向上_任意_任意_0")
print(s1)

# s2 = Signal(k1="60分钟", k2="D1", k3="表里关系V230101", v1="向上", v2="任意", v3="任意", v4="0")
# print(s2)

# s3 = Signal("30分钟_D1_表里关系V230101_向上_任意_任意_0")
# print(s3)

Signal('30分钟_D1_表里关系V230101_向上_任意_任意_0')


### 测试 Event 对象
---

In [ ]:
from rs_czsc import Event, Signal


In [14]:
from rs_czsc import Operate

In [19]:
Operate.lo

<function Operate.lo()>